# FPUS23 — YOLO SOTA (Despeckle + Optimized)

- Despeckle ultrasound images (median blur, k=5) across train/val/test.
- Train YOLO11 n/s/m with medical‑friendly augs, cosine LR.
- Switch train to balanced‑despeckled; keep val/test on despeckled originals.
- Stage‑2 resume at 896 for the small model.
- Save every epoch to Drive and auto‑resume.


In [ ]:
%pip -q install -U ultralytics opencv-python-headless
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
REPO_URL = os.environ.get('FPUS23_REPO_URL', 'https://github.com/Srinivas-Raghav-VC/MultiFetalOrgan-Detection.git')
REPO_DIR = '/content/fpus23'
if not os.path.isdir(REPO_DIR):
    !git clone $REPO_URL $REPO_DIR
else:
    !git -C $REPO_DIR fetch && git -C $REPO_DIR pull --ff-only
%cd /content/fpus23
os.makedirs('/content/drive/MyDrive/FPUS23_runs', exist_ok=True)


In [ ]:
# Prepare despeckled YOLO dataset (idempotent)
from pathlib import Path
ORIG_YAML = Path('/content/fpus23_project/dataset/fpus23_yolo/data.yaml')
IN_IMAGES = Path('/content/fpus23_project/dataset/fpus23_yolo/images')
OUT_IMAGES = Path('/content/fpus23_project/dataset/fpus23_yolo_despeckled/images')
DESP_YAML = OUT_IMAGES.parent / 'data.yaml'
if not DESP_YAML.exists():
    assert ORIG_YAML.exists(), 'Original YOLO data.yaml missing; run prepare first.'
    !python '/content/fpus23/scripts/preprocess_fpus23_despeckle.py' \
+      --input-dir $IN_IMAGES \
+      --output-dir $OUT_IMAGES \
+      --kernel-size 5 \
+      --splits train val test \
+      --data-yaml $ORIG_YAML
print('Using despeckled YAML:', DESP_YAML)


In [ ]:
# Build balanced-despeckled train split and YOLO data.yaml (val/test remain original-despeckled).
# 1) Create balanced COCO train.json (images_balanced/train + train_balanced.json)
!python '/content/fpus23/scripts/balance_fpus23_dataset.py' --coco-root '/content/fpus23_project/dataset/fpus23_coco'
# 2) Convert balanced COCO to YOLO using despeckled images as the source
from pathlib import Path
BAL_COCO = Path('/content/fpus23_project/dataset/fpus23_coco/train_balanced.json')
DESP_IMG_TRAIN = Path('/content/fpus23_project/dataset/fpus23_yolo_despeckled/images/train')
OUT_YOLO_BAL = Path('/content/fpus23_project/dataset/fpus23_yolo_balanced_despeckled')
OUT_YOLO_BAL.mkdir(parents=True, exist_ok=True)
!python '/content/fpus23/scripts/tools/coco_to_yolo.py' \
+       --coco-json $BAL_COCO \
+       --images-dir $DESP_IMG_TRAIN \
+       --out-yolo-root $OUT_YOLO_BAL \
+       --orig-data-yaml $DESP_YAML
DESP_BAL_YAML = OUT_YOLO_BAL / 'data.yaml'
print('Using balanced-despeckled YAML:', DESP_BAL_YAML)


In [ ]:
# Train YOLO11 n/s/m (balanced-despeckled + optimized). Per-epoch save and auto-resume.
sizes = ['n','s','m']
batches = {'n':16,'s':16,'m':8}
for sz in sizes:
  run = f'fpus23_yolo_sota_{sz}'
  !yolo train data=$DESP_BAL_YAML model=yolo11{sz}.pt epochs=60 batch={batches[sz]} imgsz=768 \
+       project='/content/drive/MyDrive/FPUS23_runs' name=$run deterministic=True workers=2 \
+       cos_lr=True cls=3.2 mosaic=0.2 close_mosaic=10 copy_paste=0.05 erasing=0.3 \
+       resume=True save_period=1 patience=20


In [ ]:
# Stage 2: resume small model at higher resolution (896) with lighter augs.
RUN='fpus23_yolo_sota_s'
LAST='/content/drive/MyDrive/FPUS23_runs/'+RUN+'/weights/last.pt'
!yolo train data=$DESP_BAL_YAML model=$LAST epochs=30 batch=8 imgsz=896 \
+       project='/content/drive/MyDrive/FPUS23_runs' name=$RUN deterministic=True workers=2 \
+       cos_lr=True cls=3.2 mosaic=0.0 copy_paste=0.0 erasing=0.2 \
+       resume=True save_period=1 patience=20


In [ ]:
# TTA evaluation on validation (flip/scale).
from pathlib import Path
BEST = Path('/content/drive/MyDrive/FPUS23_runs/fpus23_yolo_sota_s/weights/best.pt')
if BEST.exists():
  !yolo val model=$BEST data=$DESP_BAL_YAML imgsz=896 batch=8 augment=True
else:
  print('best.pt not found yet.')


In [ ]:
# Compact summary table
import pandas as pd
from pathlib import Path
BASE = Path('/content/drive/MyDrive/FPUS23_runs')
runs = [f'fpus23_yolo_sota_{k}' for k in ['n','s','m']]
rows = []
for r in runs:
  p = BASE / r / 'results.csv'
  if p.exists():
    df = pd.read_csv(p).dropna(how='all')
    if len(df):
      last = df.tail(1).squeeze()
      rows.append({'run': r, 'epoch': last.get('epoch'),
                   'mAP50': last.get('metrics/mAP50(B)'),
                   'mAP50-95': last.get('metrics/mAP50-95(B)'),
                   'precision': last.get('metrics/precision(B)'),
                   'recall': last.get('metrics/recall(B)')})
if rows:
  display(pd.DataFrame(rows).sort_values('mAP50-95', ascending=False))
else:
  print('No results found yet.')
